<a href="https://colab.research.google.com/github/CarlosfcPinheiro/pibic-api-llm-integration/blob/main/pibic_aplicacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Instalação das dependências ==============
!pip install fastapi uvicorn pyngrok nest-asyncio

In [3]:
# Definindo secret keys e variáveis globais ====================
from google.colab import userdata

HUGGINGFACE_API_URL = "https://api-inference.huggingface.co/models/facebook/bart-large-cnn"
HUGGINGFACE_TOKEN = userdata.get('HUGGINGFACE_TOKEN');
NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN');

In [4]:
# Criação da API com FastAPI ==============
from fastapi import FastAPI, Request
# permite a execução de próximos
import nest_asyncio
from pyngrok import ngrok
import uvicorn
from uvicorn import Config, Server

import requests

app = FastAPI()

@app.get("/")
async def root():
  return {"message": "Olá, FastAPI para o pibic!"}

@app.post("/summarize")
async def summarize(request: Request):
  body = await request.json()
  text = body.get("text", "")
  headers = {"Authorization": f"Bearer {HUGGINGFACE_TOKEN}"}

  response = requests.post(HUGGINGFACE_API_URL, headers=headers, json={"inputs": text})
  result = response.json()

  return {"summary": result[0]["summary_text"]}

In [5]:
# Permite que aplicações assíncronas rodem no ambiente do colab ==============
nest_asyncio.apply()

In [6]:
# Conectar a instância do ngrok =================================
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
public_url = ngrok.connect(8000)
print(public_url)

NgrokTunnel: "https://zetta-faunlike-kindheartedly.ngrok-free.dev" -> "http://localhost:8000"


In [7]:
# Configurações do servidor uvicorn ======================
config = Config(app=app, host="0.0.0.0", port=8000, log_level="info")
server = Server(config=config)

In [8]:
# Inicializando o servidor =================
import asyncio
await server.serve()

INFO:     Started server process [1129]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     200.124.166.185:0 - "GET / HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [1129]
